<div class="alert alert-info">

**Note:**    WARNING: This notebook uses a new, early release Chameleon python API. The notebook and API will likely be updated without notice. To request a new python feature, please create a help desk ticket. 
    

</div>

# Create a Ryu OpenFlow Controller

In [ ]:
import json
import os
import chi

from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *

#Config with your project and site
#chi.set('project_name', 'CH-816532') # Replace with your project name
chi.set('project_name', 'ExoGENI@Chameleon') # Replace with your project name
chi.set('region_name', 'CHI@UC')     # Optional, defaults to 'CHI@UC'

#Insert keypair name
key_name = 'pruth-jupyter'  # Change to your keypair

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
server_name = username+'RyuController'
lease_name = username+'RyuControllerLease'

#Server attributes
image_name='CC-CentOS7'
flavor_name='baremetal'
node_type="compute_skylake"
server_count=1
network_name='sharednet1'

In [ ]:
# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_fip_reservation(reservation_list, count=server_count)

# Create the lease
chi.blazar().lease.create(name=lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

In [ ]:
#Get the lease by name
lease = get_lease_by_name(lease_name)
    
#Print the lease info
print(json.dumps(lease, indent=2))

In [ ]:
#Get the lease by name
lease = get_lease_by_name(lease_name)

compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', lease['reservations']))[0]['id']
floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', lease['reservations']))[0]['id']

print("compute_reservation_id: " + compute_reservation_id)
print("floatingip_reservation_id: " + floatingip_reservation_id)

In [ ]:
#create the server
server = create_server(server_name, 
                       reservation_id=compute_reservation_id, 
                       key_name=key_name, 
                       network_name=network_name, 
                       image_name=image_name, 
                       flavor_name=flavor_name)


In [ ]:
floating_ip = associate_floating_ip(server_name)

print('Floating IP: ' + str(floating_ip))

In [ ]:
ryu_app='simple_switch_13_custom_chameleon.py'
ryu_port='6653'
mirror_port=''

script= '#!/bin/bash'   '\n' \
    '{'   '\n' \
    'yum install -y yum-utils device-mapper-persistent-data lvm2 vim'   '\n' \
    'yum-config-manager --add-repo https://download.docker.com/linux/centos/docker-ce.repo'   '\n' \
    'yum install -y docker-ce'   '\n' \
    'systemctl start docker'   '\n' \
    '   \n' \
    r'RECIPE_REPO=\"https://github.com/RENCI-NRIG/exogeni-recipes.git\"'   '\n' \
    'RECIPE_DIR=\"/opt/exogeni-recipes\"'   '\n' \
    'RECIPE_APP=\"openflow-controller/docker\"'   '\n' \
    'DOCKER_IMAGE=\"centos-ryu\"'   '\n' \
    'DOCKER_CONTAINER_NAME=\"ryu-controller\"'   '\n' \
    'OFP_TCP_LISTEN_PORT=\"'+ryu_port+'\"'   '\n' \
    'RYU_APP=\"/opt/ryu_app/'+ryu_app+'\"'   '\n' \
    'MIRROR_PORT='+mirror_port+   '\n' \
    '   \n' \
    'git clone  --no-checkout \${RECIPE_REPO} \${RECIPE_DIR}'   '\n' \
    'cd \${RECIPE_DIR} && git config core.sparsecheckout true'   '\n' \
    'echo \"\${RECIPE_APP}/*\" >> .git/info/sparse-checkout'   '\n' \
    'git read-tree -m -u HEAD'   '\n' \
    '   \n' \
    'cd \${RECIPE_DIR}/\${RECIPE_APP}'   '\n' \
    'docker volume create var_run_ryu'   '\n' \
    'docker volume create var_log_ryu'   '\n' \
    'docker volume create opt_ryu'   '\n' \
    'docker volume create opt_ryu_chameleon'   '\n' \
    "sed -r -i \'s/^(RYU_APP=.*)/#\1/g\' ryu_start.sh"   '\n' \
    "sed -r -i \'s/^(OFP_TCP_LISTEN_PORT=.*)/#\1/g\' ryu_start.sh"   '\n' \
    '   \n' \
    'docker build -t \${DOCKER_IMAGE} .'   '\n' \
    '   \n' \
    'docker run --rm -dit -p \${OFP_TCP_LISTEN_PORT}:\${OFP_TCP_LISTEN_PORT} -p 8080:8080 -v opt_ryu_chameleon:/opt/ryu_chameleon -v opt_ryu:/opt/ryu -v var_log_ryu:/var/log/ryu -v var_run_ryu:/var/run/ryu -e RYU_APP=\${RYU_APP} -e OFP_TCP_LISTEN_PORT=\${OFP_TCP_LISTEN_PORT}  --name=\${DOCKER_CONTAINER_NAME} \${DOCKER_IMAGE}'   '\n' \
    '   \n' \
    'echo done'   '\n' \
    '   \n' \
    '} > /tmp/boot.log 2>&1'   '\n' 

In [ ]:
import paramiko 
#from scp import SCPClient, SCPException

key = paramiko.RSAKey.from_private_key_file("/home/pruth/work/pruth-jupyter")
client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())

client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(floating_ip,username='cc',pkey = key)

stdin, stdout, stderr = client.exec_command('echo \"' + script + '\" > script.sh; chmod +x script.sh; sudo ./script.sh')
print (stdout.read())
print (stderr.read())

client.close()
print ('done!')

In [ ]:
delete_server_by_name(server_name)
delete_lease_by_name(lease_name)